# **CSCE 5218 / CSCE 4930 Deep Learning**

# **HW1a The Perceptron** (20 pt)


In [1]:
# Get the datasets
!wget http://www.cse.unt.edu/~blanco/csce5218/hw1a/train.dat
!wget http://www.cse.unt.edu/~blanco/csce5218/hw1a/test.dat

--2022-02-07 02:00:59--  http://www.cse.unt.edu/~blanco/csce5218/hw1a/train.dat
Resolving www.cse.unt.edu (www.cse.unt.edu)... 129.120.151.91
Connecting to www.cse.unt.edu (www.cse.unt.edu)|129.120.151.91|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 11244 (11K) [application/x-ns-proxy-autoconfig]
Saving to: ‘train.dat’

train.dat           100%[===================>]  10.98K  --.-KB/s    in 0s      

2022-02-07 02:00:59 (197 MB/s) - ‘train.dat’ saved [11244/11244]

--2022-02-07 02:00:59--  http://www.cse.unt.edu/~blanco/csce5218/hw1a/test.dat
Resolving www.cse.unt.edu (www.cse.unt.edu)... 129.120.151.91
Connecting to www.cse.unt.edu (www.cse.unt.edu)|129.120.151.91|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2844 (2.8K) [application/x-ns-proxy-autoconfig]
Saving to: ‘test.dat’

test.dat            100%[===================>]   2.78K  --.-KB/s    in 0s      

2022-02-07 02:00:59 (370 MB/s) - ‘test.dat’ saved [2844/2844]



In [2]:
# Take a peek at the datasets
!head train.dat
!head test.dat

A1	A2	A3	A4	A5	A6	A7	A8	A9	A10	A11	A12	A13	
1	1	0	0	0	0	0	0	1	1	0	0	1	0
0	0	1	1	0	1	1	0	0	0	0	0	1	0
0	1	0	1	1	0	1	0	1	1	1	0	1	1
0	0	1	0	0	1	0	1	0	1	1	1	1	0
0	1	0	0	0	0	0	1	1	1	1	1	1	0
0	1	1	1	0	0	0	1	0	1	1	0	1	1
0	1	1	0	0	0	1	0	0	0	0	0	1	0
0	0	0	1	1	0	1	1	1	0	0	0	1	0
0	0	0	0	0	0	1	0	1	0	1	0	1	0
A1	A2	A3	A4	A5	A6	A7	A8	A9	A10	A11	A12	A13
1	1	1	1	0	0	1	1	0	0	0	1	1	0
0	0	0	1	0	0	1	1	0	1	0	0	1	0
0	1	1	1	0	1	1	1	1	0	0	0	1	0
0	1	1	0	1	0	1	1	1	0	1	0	1	0
0	1	0	0	0	1	0	1	0	1	0	0	1	0
0	1	1	0	0	1	1	1	1	1	1	0	1	0
0	1	1	1	0	0	1	1	0	0	0	1	1	0
0	1	0	0	1	0	0	1	1	0	1	1	1	0
1	1	1	1	0	0	1	1	0	0	0	0	1	0


### Build the Perceptron Model

You will need to complete some of the function definitions below.  DO NOT import any other libraries to complete this. 

In [3]:
import math
import itertools
import re


# Corpus reader, all columns but the last one are coordinates;
#   the last column is the label
def read_data(file_name):
    f = open(file_name, 'r')

    data = []
    # Discard header line
    f.readline()
    for instance in f.readlines():
        if not re.search('\t', instance): continue
        instance = list(map(int, instance.strip().split('\t')))
        # Add a dummy input so that w0 becomes the bias
        instance = [-1] + instance
        data += [instance]
    return data


def dot_product(array1, array2):
    #TODO: Return dot product of array 1 and array 2
    dot_prod = sum([i * j for i, j in zip(array1, array2)])
    return dot_prod


def sigmoid(x):
    #TODO: Return outpout of sigmoid function on x

    sigmoid_function = (1 / (1 + math.exp(-x)))
    return sigmoid_function

# The output of the model, which for the perceptron is 
# the sigmoid function applied to the dot product of 
# the instance and the weights
def output(weight, instance):
    #TODO: return the output of the model 
    return sigmoid(dot_product(weight,instance))

# Predict the label of an instance; this is the definition of the perceptron
# you should output 1 if the output is >= 0.5 else output 0
def predict(weights, instance):
    #TODO: return the prediction of the model
    predict_value = output(weights, instance)
    if(predict_value>=0.5):
      return 1
    else:
      return 0



# Accuracy = percent of correct predictions
def get_accuracy(weights, instances):
    # You do not to write code like this, but get used to it
    correct = sum([1 if predict(weights, instance) == instance[-1] else 0
                   for instance in instances])
    return correct * 100 / len(instances)


# Train a perceptron with instances and hyperparameters:
#       lr (learning rate) 
#       epochs
# The implementation comes from the definition of the perceptron
#
# Training consists on fitting the parameters which are the weights
# that's the only thing training is responsible to fit
# (recall that w0 is the bias, and w1..wn are the weights for each coordinate)
#
# Hyperparameters (lr and epochs) are given to the training algorithm
# We are updating weights in the opposite direction of the gradient of the error,
# so with a "decent" lr we are guaranteed to reduce the error after each iteration.
def train_perceptron(instances, lr, epochs):

    #TODO: name this step
    # Initialising weights to "0" with the length of number of input parameters
    weights = [0] * (len(instances[0])-1)

    for _ in range(epochs):
        for instance in instances:
            #TODO: name these steps
            # Assigning (Calculating) the values to the neurons value
            in_value = dot_product(weights, instance)
            output = sigmoid(in_value)
            error = instance[-1] - output
            #TODO: name these steps
            # Updating the weights based on the error value
            for i in range(0, len(weights)):
                weights[i] += lr * error * output * (1-output) * instance[i]

    return weights

## Run it

In [4]:
instances_tr = read_data("train.dat")
instances_te = read_data("test.dat")
lr = 0.005
epochs = 5
weights = train_perceptron(instances_tr, lr, epochs)
accuracy = get_accuracy(weights, instances_te)
print(f"#tr: {len(instances_tr):3}, epochs: {epochs:3}, learning rate: {lr:.3f}; "
      f"Accuracy (test, {len(instances_te)} instances): {accuracy:.1f}")

#tr: 400, epochs:   5, learning rate: 0.005; Accuracy (test, 100 instances): 68.0


## Questions

Answer the following questions. Include your implementation and the output for each question.



### Question 1

In `train_perceptron(instances, lr, epochs)`, we have the follosing code:
```
in_value = dot_product(weights, instance)
output = sigmoid(in_value)
error = instance[-1] - output
```

Why don't we have the following code snippet instead?
```
output = predict(weights, instance)
error = instance[-1] - output
```

#### TODO Add your answer here (text only)

Answer:<br>
If the code snippet is changed(i.e., dot_product is removed ), the weights will not be updated, resulting in the model being unable to distinguish between two classes which results in the  poor accuracy. The activation function,gives the  nonlinearity in the data, generates the gradients for updating weights during back propagation, and aids the model in learning more complicated patterns.







### Question 2
Train the perceptron with the following hyperparameters and calculate the accuracy with the test dataset.

```
tr_percent = [5, 10, 25, 50, 75, 100] # percent of the training dataset to train with
num_epochs = [5, 10, 20, 50, 100]              # number of epochs
lr = [0.005, 0.01, 0.05]              # learning rate
```

TODO: Write your code below and include the output at the end of each training loop (NOT AFTER EACH EPOCH)
of your code.The output should look like the following:
```
# tr:  20, epochs:   5, learning rate: 0.005; Accuracy (test, 100 instances): 68.0
# tr:  20, epochs:  10, learning rate: 0.005; Accuracy (test, 100 instances): 68.0
# tr:  20, epochs:  20, learning rate: 0.005; Accuracy (test, 100 instances): 68.0
[and so on for all the combinations]
```
You will get different results with different hyperparameters.

#### TODO Add your answer here (code and output in the format above) 



In [6]:
instances_tr = read_data("train.dat")
instances_te = read_data("test.dat")
tr_percent = [5, 10, 25, 50, 75, 100] # percent of the training dataset to train with
num_epochs = [5, 10, 20, 50, 100]     # number of epochs
lr_array = [0.005, 0.01, 0.05]        # learning rate

for lr in lr_array:
  for tr_size in tr_percent:
    for epochs in num_epochs:
      size =  round(len(instances_tr)*tr_size/100)
      pre_instances = instances_tr[0:size]
      weights = train_perceptron(pre_instances, lr, epochs)
      accuracy = get_accuracy(weights, instances_te)
      print(f"#tr: {len(pre_instances):0}, epochs: {epochs:3}, learning rate: {lr:.3f}; "
            f"Accuracy (test, {len(instances_te)} instances): {accuracy:.1f}")

#tr: 20, epochs:   5, learning rate: 0.005; Accuracy (test, 100 instances): 68.0
#tr: 20, epochs:  10, learning rate: 0.005; Accuracy (test, 100 instances): 68.0
#tr: 20, epochs:  20, learning rate: 0.005; Accuracy (test, 100 instances): 68.0
#tr: 20, epochs:  50, learning rate: 0.005; Accuracy (test, 100 instances): 68.0
#tr: 20, epochs: 100, learning rate: 0.005; Accuracy (test, 100 instances): 68.0
#tr: 40, epochs:   5, learning rate: 0.005; Accuracy (test, 100 instances): 68.0
#tr: 40, epochs:  10, learning rate: 0.005; Accuracy (test, 100 instances): 68.0
#tr: 40, epochs:  20, learning rate: 0.005; Accuracy (test, 100 instances): 68.0
#tr: 40, epochs:  50, learning rate: 0.005; Accuracy (test, 100 instances): 68.0
#tr: 40, epochs: 100, learning rate: 0.005; Accuracy (test, 100 instances): 68.0
#tr: 100, epochs:   5, learning rate: 0.005; Accuracy (test, 100 instances): 68.0
#tr: 100, epochs:  10, learning rate: 0.005; Accuracy (test, 100 instances): 68.0
#tr: 100, epochs:  20, lea

### Question 3
Write a couple paragraphs interpreting the results with all the combinations of hyperparameters. Drawing a plot will probably help you make a point. In particular, answer the following:
- A. Do you need to train with all the training dataset to get the highest accuracy with the test dataset?
- B. How do you justify that training the second run obtains worse accuracy than the first one (despite the second one uses more training data)?
   ```
#tr: 100, epochs:  20, learning rate: 0.050; Accuracy (test, 100 instances): 71.0
#tr: 200, epochs:  20, learning rate: 0.005; Accuracy (test, 100 instances): 68.0
```
- C. Can you get higher accuracy with additional hyperparameters (higher than `80.0`)?
- D. Is it always worth training for more epochs (while keeping all other hyperparameters fixed)?

#### TODO: Add your answer here (code and text)



Answer (3) :<br>
**3a Question:**<br>
Do you need to train with all the training dataset to get the highest accuracy with the test dataset? <br>
**3a Ans:** <br>
tr: 100, epochs: 100, learning rate: 0.005; Accuracy (test, 100 instances): 68.0<br>
tr: 200, epochs: 100, learning rate: 0.005; Accuracy (test, 100 instances): 74.0
<br>

In the above example we can observe that as the training data increased from the 100 to 200 keeping all the hyperparameters constant the accuracy of the model has been increased. i.e., by providing more data to the model helps the extarct more insights from the data.
<br>
<br>
**3b Question**<br>
How do you justify that training the second run obtains worse accuracy than the first one (despite the second one uses more training data)?<br>
tr: 100, epochs:  20, learning rate: 0.050; Accuracy (test, 100 instances): 71.0
<br>
tr: 200, epochs:  20, learning rate: 0.005; Accuracy (test, 100 instances): 68.0
<br>
**3b Answer**<br>
In the above scenario, although we provide the training data more compared to the previous set, the accuracy not only depends on the amount of data but it also depends on the other hyperparameters as well in the above example the learning rate of tr:100 is 0.05 but for tr:200 it is 0.005 where learning rate indicates how much the model need to correct from the past value. So, here the learning rate for 100 is greater than tr 200 so the model is learning less compared to training data 100 which resulted in the less accuracy.

**3c Question**<br>
C. Can you get higher accuracy with additional hyperparameters (higher than 80.0)?<br>
**3c Answer**<br>
It is possible to get higher accuracy with additional hyperparameter depending on the search space we are looking. I we can look at the right search space we can get the higher accuracy with less hyperparameters as well.
<br>**3d Question**<br>
Is it always worth training for more epochs (while keeping all other hyperparameters fixed)?<br>
**3d Answer**<br>
Yes,it is worth to train the data with more epoch until the model gets overfit, we can see that increase in the accuracy with the increase in the epochs untill it get overfit. Once the model gets overfit the increasing the epochs will result in the poor accuracy compared to previous epochs